In [2]:
import ast
import pandas as pd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import numpy as np
from shapely.geometry.polygon import LinearRing, Polygon
import matplotlib.pyplot as plt
import geopandas as gpd

In [3]:
df = pd.read_csv('housedata.csv')

In [4]:
dict = df.hdpData[0]
result = ast.literal_eval(dict)
hdpdf = pd.DataFrame(result['homeInfo'])
for i in range(1,len(df.hdpData)):
    dict = df.hdpData[i]
    result = ast.literal_eval(dict)
    dfholder = pd.DataFrame(result['homeInfo'])
    if len(dfholder) > 1:
        dfholder = dfholder.loc[['is_FSBA']]
    dfholder = dfholder.astype({'isFeatured':bool,'shouldHighlight':bool,
                                'listing_sub_type':bool, 'isUnmappable':bool,
                                'isPreforeclosureAuction':bool, 'isNonOwnerOccupied':bool,
                                'isPremierBuilder':bool,'isZillowOwned':bool
                                })
    hdpdf = pd.concat([hdpdf,dfholder])

In [5]:
hdpdf.reset_index(inplace=True,drop = True)

In [24]:
newhdpdf = hdpdf[['zpid', 'streetAddress', 'zipcode', 'city', 'state', 'latitude',
'longitude', 'price', 'bathrooms', 'bedrooms', 'livingArea', 'homeType', 'priceForHDP', 'taxAssessedValue', 'lotAreaValue']]

In [7]:
newdf = df[['id','imgSrc','hasImage','detailUrl', 'statusType','statusText','price','unformattedPrice', 'address',
'addressStreet', 'addressCity', 'addressZipcode', 'isUndisclosedAddress', 'beds', 'baths', 'area', 'brokerName'
]]

In [28]:
newdf.to_csv('cleanhousedata.csv', index = False)
newhdpdf.to_csv('cleanhdpdata.csv', index = False)

In [ ]:
df = pd.read_csv('finaldataset.csv')

In [ ]:
df.drop('priceForHDP', axis = 1, inplace = True)

In [ ]:
# adapted from https://medium.com/python-in-plain-english/mapping-with-pythons-geopandas-2869bb758b08

bnd_gdf = gpd.read_file('local-area-boundary/local-area-boundary.shp')
ps_gdf = gpd.read_file('public-streets/public-streets.shp')

poly = bnd_gdf['geometry']

locations = poly.apply(lambda x: x.representative_point().coords[:])
locations = [coords[0] for coords in locations]

# x,y = poly.exterior[0].xy

fig, ax = plt.subplots(1, figsize = (30,12))

bnd_gdf.plot(ax = ax, color = 'black')
ps_gdf.plot(ax = ax, color = 'white', alpha = 0.4)
ax.axis('off')

for i in range(len(poly) - 1):
        x,y = poly.exterior[i].xy
        plt.plot(x, y, color='#eb0707', alpha=0.7,
            linewidth=3, solid_capstyle='round', zorder=2)
        plt.annotate(text = bnd_gdf['name'][i], xy = locations[i], c = 'yellow',
                 horizontalalignment='center', fontsize = 10)

In [ ]:
newaddressList = []

for i in range(len(df['streetAddress'])):
    textaddress = df['streetAddress'].iloc[i]
    newAddress = textaddress.split('#', 1)[0]
    newAddress = newAddress.split('PENTHOUSE', 1)[0]
    newaddressList.append(newAddress)

df['newAddress'] = newaddressList

In [ ]:
unknown_address = []

for i in range(0,300):
    y,x = df['latitude'].iloc[i],df['longitude'].iloc[i]

    if np.isnan(y) == True or np.isnan(x) == True:
        address = df['newAddress'].iloc[i]+ ' ' + df['city'].iloc[i]+ ' ' + df['state'].iloc[i]
        geolocator = Nominatim(user_agent="Barry")
        location = geolocator.geocode(address)
        try:
            df.at[i,'latitude'] = location.latitude
            df.at[i,'longitude'] = location.longitude
        except:
            unknown_address.append(df['newAddress'].iloc[i])

In [ ]:
for i in range(300, 338):
    y,x = df['latitude'].iloc[i],df['longitude'].iloc[i]

    if np.isnan(y) == True or np.isnan(x) == True:
        address = df['newAddress'].iloc[i]+ ' ' + df['city'].iloc[i]+ ' ' + df['state'].iloc[i]
        geolocator = Nominatim(user_agent="Barry")
        location = geolocator.geocode(address)
        try:
            df.at[i,'latitude'] = location.latitude
            df.at[i,'longitude'] = location.longitude
        except:
            unknown_address.append(df['newAddress'].iloc[i])

In [ ]:
for i in range(339, 553):
    y,x = df['latitude'].iloc[i],df['longitude'].iloc[i]

    if np.isnan(y) == True or np.isnan(x) == True:
        address = df['newAddress'].iloc[i]+ ' ' + df['city'].iloc[i]+ ' ' + df['state'].iloc[i]
        geolocator = Nominatim(user_agent="Barry")
        location = geolocator.geocode(address)
        try:
            df.at[i,'latitude'] = location.latitude
            df.at[i,'longitude'] = location.longitude
        except:
            unknown_address.append(df['newAddress'].iloc[i])

In [ ]:
for i in range(554, 800):
    y,x = df['latitude'].iloc[i],df['longitude'].iloc[i]

    if np.isnan(y) == True or np.isnan(x) == True:
        address = df['newAddress'].iloc[i]+ ' ' + df['city'].iloc[i]+ ' ' + df['state'].iloc[i]
        geolocator = Nominatim(user_agent="Barry")
        location = geolocator.geocode(address)
        try:
            df.at[i,'latitude'] = location.latitude
            df.at[i,'longitude'] = location.longitude
        except:
            unknown_address.append(df['newAddress'].iloc[i])

In [ ]:
messed_up_latitude = 49.2823281
messed_up_longitude = -123.1021128
i = 338
df.at[i,'latitude'] = messed_up_latitude
df.at[i,'longitude'] = messed_up_longitude
i = 553
df.at[i,'latitude'] = messed_up_latitude
df.at[i,'longitude'] = messed_up_longitude

In [ ]:
# https://stackoverflow.com/questions/43892459/check-if-geo-point-is-inside-or-outside-of-polygon

# checking every latitude and longitude to determine what neighbourhood house resides in
neighbourhood_name = []
for i in range(len(df['latitude'])):
    checker = 0
    y,x = df['latitude'].iloc[i],df['longitude'].iloc[i]

    if np.isnan(y) == True or np.isnan(x) == True:
        neighbourhood_name.append(np.nan)

    else:
        for name, poly in zip(bnd_gdf['name'],bnd_gdf['geometry']):
            point = Point(x,y)
            if poly.contains(point) == True:
                neighbourhood_name.append(name)
                checker = 1
        if checker != 1:
            neighbourhood_name.append(np.nan)

In [ ]:
df['neighbourhoodName'] = neighbourhood_name

In [ ]:
# df.to_csv('reviseddataset.csv', index = False)

In [ ]:
df = pd.read_csv('reviseddataset.csv')

In [ ]:
# checking where latitude is missing
# df[df.latitude.isna()]

In [ ]:
# check to see which columns had NAs
# df.isna().sum()

In [ ]:
coalharbourlat = 49.2918062
coalharbourlong = -123.1285668
df.at[581,'streetAddress'] = '1550 Coal Harbour Quay #L401'
df.at[581,'newAddress'] = '1550 Coal Harbour Quay'
df.at[581,'latitude'] = coalharbourlat
df.at[581,'longitude'] = coalharbourlong

In [ ]:
hastingslat = 49.2814934
hastingslong = -123.09566373306427
df.at[789,'streetAddress'] = '383 East Hastings St'
df.at[789,'newAddress'] = '383 East Hastings St'
df.at[789,'latitude'] = hastingslat
df.at[789,'longitude'] = hastingslong

In [ ]:
df = df.drop(index = 427)

In [ ]:
df.at[481,'livingArea'] = 3712.0

In [ ]:
df.at[(105,483,674),'lotAreaValue'] = 0.0

In [ ]:
df.at[105,'brokerName'] = 'Oakwyn Realty Ltd. Brokerage'
df.at[481,'brokerName'] = 'ICI Source Real Asset Services Inc. Brokerage'
df.at[483,'brokerName'] = 'Engel & Volkers Vancouver Brokerage'
df.at[674,'brokerName'] = 'Reign Realty Brokerage'


In [ ]:
missingNeighbourname = df[df.neighbourhoodName.isna()]

In [ ]:
bnd_gdf = gpd.read_file('local-area-boundary/local-area-boundary.shp')
ps_gdf = gpd.read_file('public-streets/public-streets.shp')

neighbourhood_name = []
for i in range(len(missingNeighbourname['latitude'])):
    checker = 0
    y,x = missingNeighbourname['latitude'].iloc[i],missingNeighbourname['longitude'].iloc[i]

    if np.isnan(y) == True or np.isnan(x) == True:
        neighbourhood_name.append(np.nan)

    else:
        for name, poly in zip(bnd_gdf['name'],bnd_gdf['geometry']):
            point = Point(x,y)
            if poly.contains(point) == True:
                neighbourhood_name.append(name)
                checker = 1
        if checker != 1:
            neighbourhood_name.append(np.nan)

In [ ]:
neighbourhood_name[0] = 'Renfrew-Collingwood'

In [ ]:
df.at[(63,581,789), 'neighbourhoodName'] = neighbourhood_name

In [ ]:
# df.to_csv('reviseddataset.csv', index = False)